### Pymongo
- /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
- pip install pymongo

In [ ]:
import pymongo

client = pymongo.MongoClient(host= '10.241.49.55', port=20011, username='dba',password='dba')
# print(client.list_database_names())
db= client['lai-alpha']

In [ ]:
import pandas as pd

# 방법1 cursor - dict list - dataframe
resourceDf = pd.DataFrame(list(db.resource.find()))
applicationDf = pd.DataFrame(list(db.application.find()))
paDf = pd.DataFrame(list(db.productApplication.find()))


serverResourceDf = resourceDf[
    resourceDf["type"].isin(['server','legacy-server','legacy-physical-server'])]

total_resource_count = serverResourceDf.shape[0]
total_server_count = db.server.count_documents({})

In [ ]:
import json

defaultPA = paDf[paDf["name"]=='default']\
default_pa_id = defaultPA['_id'].loc[0]
print(default_pa_id)